In [1]:
import tensorflow as tf
from data_utility import *

iterations      = 200
batch_size      = 250
total_epoch     = 164
weight_decay    = 0.0003
dropout_rate    = 0.5
momentum_rate   = 0.9
log_save_path   = './vgg_logs'
model_save_path = './model/'

/home/du0/15CS30043/anaconda3/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# ========================================================== #
# ├─ bias_variable()
# ├─ conv2d()           With Batch Normalization
# ├─ max_pool()
# └─ global_avg_pool()
# ========================================================== #


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape, dtype=tf.float32 )
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def max_pool(input, k_size=1, stride=1, name=None):
    return tf.nn.max_pool(input, ksize=[1, k_size, k_size, 1], strides=[1, stride, stride, 1], padding='SAME',name=name)

def batch_norm(input):
    return tf.contrib.layers.batch_norm(input, decay=0.9, center=True, scale=True, epsilon=1e-3, is_training=train_flag, updates_collections=None)

In [3]:
# ========================================================== #
# ├─ _random_crop() 
# ├─ _random_flip_leftright()
# ├─ data_augmentation()
# ├─ data_preprocessing()
# └─ learning_rate_schedule()
# ========================================================== #

def _random_crop(batch, crop_shape, padding=None):
        oshape = np.shape(batch[0])
        
        if padding:
            oshape = (oshape[0] + 2*padding, oshape[1] + 2*padding)
        new_batch = []
        npad = ((padding, padding), (padding, padding), (0, 0))
        for i in range(len(batch)):
            new_batch.append(batch[i])
            if padding:
                new_batch[i] = np.lib.pad(batch[i], pad_width=npad,
                                          mode='constant', constant_values=0)
            nh = random.randint(0, oshape[0] - crop_shape[0])
            nw = random.randint(0, oshape[1] - crop_shape[1])
            new_batch[i] = new_batch[i][nh:nh + crop_shape[0],
                                        nw:nw + crop_shape[1]]
        return new_batch

def _random_flip_leftright(batch):
        for i in range(len(batch)):
            if bool(random.getrandbits(1)):
                batch[i] = np.fliplr(batch[i])
        return batch

def data_preprocessing(x_train,x_test):

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')

    x_train[:,:,:,0] = (x_train[:,:,:,0] - np.mean(x_train[:,:,:,0])) / np.std(x_train[:,:,:,0])
    x_train[:,:,:,1] = (x_train[:,:,:,1] - np.mean(x_train[:,:,:,1])) / np.std(x_train[:,:,:,1])
    x_train[:,:,:,2] = (x_train[:,:,:,2] - np.mean(x_train[:,:,:,2])) / np.std(x_train[:,:,:,2])

    x_test[:,:,:,0] = (x_test[:,:,:,0] - np.mean(x_test[:,:,:,0])) / np.std(x_test[:,:,:,0])
    x_test[:,:,:,1] = (x_test[:,:,:,1] - np.mean(x_test[:,:,:,1])) / np.std(x_test[:,:,:,1])
    x_test[:,:,:,2] = (x_test[:,:,:,2] - np.mean(x_test[:,:,:,2])) / np.std(x_test[:,:,:,2])

    return x_train, x_test

def learning_rate_schedule(epoch_num):
      if epoch_num < 81:
          return 0.1
      elif epoch_num < 121:
          return 0.01
      else:
          return 0.001

def data_augmentation(batch):
    batch = _random_flip_leftright(batch)
    batch = _random_crop(batch, [32,32], 4)
    return batch

def run_testing(sess,ep,k_list):
    acc = 0.0
    loss = 0.0
    pre_index = 0
    add = 1000
    for it in range(10):
        batch_x = test_x[pre_index:pre_index+add]
        batch_y = test_y[pre_index:pre_index+add]
        pre_index = pre_index + add
        loss_, acc_  = sess.run([cross_entropy,accuracy],feed_dict={x:batch_x, y_:batch_y, keep_prob: 1.0, train_flag: False, K:k_list})
        loss += loss_ / 10.0
        acc += acc_ / 10.0
    summary = tf.Summary(value=[tf.Summary.Value(tag="test_loss", simple_value=loss), 
                            tf.Summary.Value(tag="test_accuracy", simple_value=acc)])
    return acc, loss, summary

In [4]:
# ========================================================== #
# ├─ main()
# Training and Testing 
# Save train/teset loss and acc for visualization
# Save Model in ./model
# ========================================================== #

#Quantization functions
G = tf.get_default_graph()
def quantize(x):
    with G.gradient_override_map({"Sign": "Identity"}):
        return tf.sign(x)


def quantize_2(x):
    with G.gradient_override_map({"Sign": "Identity", "Where":"Identity", "OnesLike":"Identity"}):
        result = tf.sign(x)*tf.where(tf.abs(x)<= 0.75, tf.ones_like(x)/2,tf.ones_like(x))
        return result

    
def quantization(x,k):
    return tf.cond(tf.equal(k,1),lambda: quantize(x),lambda: quantize_2(x))    


#Calculating Regularization Loss
def reg_1(x):
    return tf.reduce_sum(tf.abs(tf.add(tf.ones_like(x),tf.negative(tf.square(x)))))
    

def reg_2(x):
    return tf.reduce_sum(tf.abs(tf.multiply(tf.add(tf.ones_like(x),tf.negative(tf.square(x))),tf.add(tf.ones_like(x)/4,tf.negative(tf.square(x))))))
    
def reg_loss_fn(x,k):
    return tf.cond(tf.equal(k,1),lambda: reg_1(x),lambda: reg_2(x))    

def mask_1(x,epsilon):
    return tf.reduce_mean(tf.abs(tf.where(tf.less(tf.abs(tf.subtract(tf.abs(x),tf.ones_like(x))),epsilon),tf.sign(x), tf.zeros_like(x))))
    

def mask_2(x,epsilon):
    return tf.reduce_mean(tf.abs(tf.where(tf.logical_or(tf.less(tf.abs(tf.subtract(tf.abs(x),tf.ones_like(x))),epsilon),tf.less(tf.abs(tf.subtract(tf.abs(x),tf.ones_like(x)/2)),epsilon)),tf.sign(x), tf.zeros_like(x))))

def mask_fn(x,k,epsilon):
    return tf.cond(tf.equal(k,1),lambda: mask_1(x,epsilon),lambda: mask_2(x,epsilon))    

def update_k(masks,k_list):
    threshold = 0.10
    for i in range(len(masks)):
        if(masks[i] < threshold):
            k_list[i] = min(k_list[i]+1,2)
    return k_list
            



In [5]:
if __name__ == '__main__':

    train_x, train_y, test_x, test_y = prepare_data()
    train_x, test_x = data_preprocessing(train_x, test_x)

    # define placeholder x, y_ , keep_prob, learning_rate
    x  = tf.placeholder(tf.float32,[None, image_size, image_size, 3])
    y_ = tf.placeholder(tf.float32, [None, class_num])
    keep_prob = tf.placeholder(tf.float32)
    learning_rate = tf.placeholder(tf.float32)
    train_flag = tf.placeholder(tf.bool)

    # build_network
    net_weights = {}
    
    num_weights = 19
    K = tf.placeholder(dtype = tf.int32, shape = (num_weights,))
    
    
    net_weights['W_conv1_1'] = {}
    net_weights['W_conv1_1']['w'] = tf.get_variable('conv1_1', shape=[3, 3, 3, 64], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv1_1 = bias_variable([64])
    W_conv1_1_sign = quantization(net_weights['W_conv1_1']['w'],K[0])
    output  = tf.nn.relu( batch_norm(conv2d(x,W_conv1_1_sign) + b_conv1_1))
    
    net_weights['W_conv1_2'] = {}
    net_weights['W_conv1_2']['w'] = tf.get_variable('conv1_2', shape=[3, 3, 64, 64], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv1_2 = bias_variable([64])
    W_conv1_2_sign = quantization(net_weights['W_conv1_2']['w'],K[1])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv1_2_sign) + b_conv1_2))
    output  = max_pool(output, 2, 2, "pool1")
    
    net_weights['W_conv2_1'] = {}
    net_weights['W_conv2_1']['w'] = tf.get_variable('conv2_1', shape=[3, 3, 64, 128], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv2_1 = bias_variable([128])
    W_conv2_1_sign = quantization(net_weights['W_conv2_1']['w'],K[2])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv2_1_sign) + b_conv2_1))
    
    net_weights['W_conv2_2'] = {}
    net_weights['W_conv2_2']['w'] = tf.get_variable('conv2_2', shape=[3, 3, 128, 128], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv2_2 = bias_variable([128])
    W_conv2_2_sign = quantization(net_weights['W_conv2_2']['w'],K[3])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv2_2_sign) + b_conv2_2))
    output  = max_pool(output, 2, 2, "pool2")
    
    net_weights['W_conv3_1'] = {}
    net_weights['W_conv3_1']['w'] = tf.get_variable('conv3_1', shape=[3, 3, 128, 256], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv3_1 = bias_variable([256])
    W_conv3_1_sign = quantization(net_weights['W_conv3_1']['w'],K[4])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv3_1_sign) + b_conv3_1))

    net_weights['W_conv3_2'] = {}
    net_weights['W_conv3_2']['w'] = tf.get_variable('conv3_2', shape=[3, 3, 256, 256], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv3_2 = bias_variable([256])
    W_conv3_2_sign = quantization(net_weights['W_conv3_2']['w'],K[5])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv3_2_sign) + b_conv3_2))

    net_weights['W_conv3_3'] = {}
    net_weights['W_conv3_3']['w'] = tf.get_variable('conv3_3', shape=[3, 3, 256, 256], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv3_3 = bias_variable([256])
    W_conv3_3_sign = quantization(net_weights['W_conv3_3']['w'],K[6])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv3_3_sign) + b_conv3_3))

    net_weights['W_conv3_4'] = {}
    net_weights['W_conv3_4']['w'] = tf.get_variable('conv3_4', shape=[3, 3, 256, 256], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv3_4 = bias_variable([256])
    W_conv3_4_sign = quantization(net_weights['W_conv3_4']['w'],K[7])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv3_4_sign) + b_conv3_4))
    output  = max_pool(output, 2, 2, "pool3")

    net_weights['W_conv4_1'] = {}
    net_weights['W_conv4_1']['w'] = tf.get_variable('conv4_1', shape=[3, 3, 256, 512], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv4_1 = bias_variable([512])
    W_conv4_1_sign = quantization(net_weights['W_conv4_1']['w'],K[8])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv4_1_sign) + b_conv4_1))

    net_weights['W_conv4_2'] = {}
    net_weights['W_conv4_2']['w'] = tf.get_variable('conv4_2', shape=[3, 3, 512, 512], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv4_2 = bias_variable([512])
    W_conv4_2_sign = quantization(net_weights['W_conv4_2']['w'],K[9])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv4_2_sign) + b_conv4_2))
    
    net_weights['W_conv4_3'] = {}
    net_weights['W_conv4_3']['w'] = tf.get_variable('conv4_3', shape=[3, 3, 512, 512], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv4_3 = bias_variable([512])
    W_conv4_3_sign = quantization(net_weights['W_conv4_3']['w'],K[10])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv4_3_sign) + b_conv4_3))
    
    net_weights['W_conv4_4'] = {}
    net_weights['W_conv4_4']['w'] = tf.get_variable('conv4_4', shape=[3, 3, 512, 512], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv4_4 = bias_variable([512])
    W_conv4_4_sign = quantization(net_weights['W_conv4_4']['w'],K[11])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv4_4_sign)) + b_conv4_4)
    output  = max_pool(output, 2, 2)
    
    net_weights['W_conv5_1'] = {}
    net_weights['W_conv5_1']['w'] = tf.get_variable('conv5_1', shape=[3, 3, 512, 512], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv5_1 = bias_variable([512])
    W_conv5_1_sign = quantization(net_weights['W_conv5_1']['w'],K[12])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv5_1_sign) + b_conv5_1))
    
    net_weights['W_conv5_2'] = {}
    net_weights['W_conv5_2']['w']= tf.get_variable('conv5_2', shape=[3, 3, 512, 512], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv5_2 = bias_variable([512])
    W_conv5_2_sign = quantization(net_weights['W_conv5_2']['w'],K[13])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv5_2_sign) + b_conv5_2))
    
    net_weights['W_conv5_3'] = {}
    net_weights['W_conv5_3']['w'] = tf.get_variable('conv5_3', shape=[3, 3, 512, 512], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv5_3 = bias_variable([512])
    W_conv5_3_sign = quantization(net_weights['W_conv5_3']['w'],K[14])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv5_3_sign) + b_conv5_3))
    
    net_weights['W_conv5_4'] = {}
    net_weights['W_conv5_4']['w'] = tf.get_variable('conv5_4', shape=[3, 3, 512, 512], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv5_4 = bias_variable([512])
    W_conv5_4_sign = quantization(net_weights['W_conv5_4']['w'],K[15])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv5_4_sign) + b_conv5_4))

    # output = tf.contrib.layers.flatten(output)
    output = tf.reshape(output,[-1,2*2*512])
    
    net_weights['W_fc1'] = {}
    net_weights['W_fc1']['w'] = tf.get_variable('fc1', shape=[2048,4096], initializer=tf.contrib.keras.initializers.he_normal())
    b_fc1 = bias_variable([4096])
    W_fc1_sign = quantization(net_weights['W_fc1']['w'],K[16])
    output = tf.nn.relu( batch_norm(tf.matmul(output,W_fc1_sign) + b_fc1) )
    output  = tf.nn.dropout(output,keep_prob)
    
    net_weights['W_fc2'] = {}
    net_weights['W_fc2']['w'] = tf.get_variable('fc7', shape=[4096,4096], initializer=tf.contrib.keras.initializers.he_normal())
    b_fc2 = bias_variable([4096])
    W_fc2_sign = quantization(net_weights['W_fc2']['w'],K[17])
    output = tf.nn.relu( batch_norm(tf.matmul(output,W_fc2_sign) + b_fc2) )
    output  = tf.nn.dropout(output,keep_prob)

    net_weights['W_fc3'] = {}
    net_weights['W_fc3']['w'] = tf.get_variable('fc3', shape=[4096,10], initializer=tf.contrib.keras.initializers.he_normal())
    b_fc3 = bias_variable([10])
    W_fc3_sign = quantization(net_weights['W_fc3']['w'],K[18])
    output = tf.nn.relu( batch_norm(tf.matmul(output,W_fc3_sign) + b_fc3) )
    # output  = tf.reshape(output,[-1,10])

    reg_loss = 0
    weights = []
    for key in net_weights.keys():
        weights.append(net_weights[key]['w'])
#     weights = [W_conv1_1,W_conv1_2,W_conv2_1,W_conv2_2,W_conv3_1,W_conv3_2,W_conv3_3,W_conv3_4,W_conv4_1,W_conv4_1,W_conv4_2,W_conv4_3,W_conv4_4,W_conv5_1,W_conv5_2,W_conv5_3,W_conv5_4,W_fc1,W_fc2,W_fc3]
    # for weight in weights:
    #     reg_loss+= tf.reduce_sum(tf.abs(tf.add(tf.ones_like(weight),tf.negative(tf.square(weight)))))
    reg_loss = reg_loss_fn(net_weights['W_conv1_1']['w'], K[0]) \
            +  reg_loss_fn(net_weights['W_conv1_2']['w'], K[1]) \
            +  reg_loss_fn(net_weights['W_conv2_1']['w'], K[2]) \
            +  reg_loss_fn(net_weights['W_conv2_2']['w'], K[3]) \
            +  reg_loss_fn(net_weights['W_conv3_1']['w'], K[4]) \
            +  reg_loss_fn(net_weights['W_conv3_2']['w'], K[5]) \
            +  reg_loss_fn(net_weights['W_conv3_3']['w'], K[6]) \
            +  reg_loss_fn(net_weights['W_conv3_4']['w'], K[7]) \
            +  reg_loss_fn(net_weights['W_conv4_1']['w'], K[8]) \
            +  reg_loss_fn(net_weights['W_conv4_2']['w'], K[9]) \
            +  reg_loss_fn(net_weights['W_conv4_3']['w'], K[10]) \
            +  reg_loss_fn(net_weights['W_conv4_4']['w'], K[11]) \
            +  reg_loss_fn(net_weights['W_conv5_1']['w'], K[12]) \
            +  reg_loss_fn(net_weights['W_conv5_2']['w'], K[13]) \
            +  reg_loss_fn(net_weights['W_conv5_3']['w'], K[14]) \
            +  reg_loss_fn(net_weights['W_conv5_4']['w'], K[15]) \
            +  reg_loss_fn(net_weights['W_fc1']['w'],K[16]) \
            +  reg_loss_fn(net_weights['W_fc2']['w'],K[17]) \
            +  reg_loss_fn(net_weights['W_fc3']['w'],K[18]) \



    # loss function: cross_entropy
    # train_step: training operation
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=output))
    l2 = tf.add_n([tf.nn.l2_loss(var) for var in tf.trainable_variables()])
    lbda = tf.placeholder(tf.float32)
    train_step = tf.train.MomentumOptimizer(learning_rate, momentum_rate,use_nesterov=True).minimize(cross_entropy + lbda*reg_loss)
    
    correct_prediction = tf.equal(tf.argmax(output,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

    #Masks
    epsilon = 0.00001
    mask = [mask_fn(weights[i],K[i],epsilon) for i in range(len(weights))]
    
    # epsilon = 0.00001
    # for i in weights:
    #     mask.append(tf.reduce_mean(tf.abs(tf.where(tf.less(tf.abs(tf.subtract(tf.abs(i),tf.ones_like(i))),epsilon),tf.sign(i), tf.zeros_like(i)))))


    #Binarization 
    binary = []
    for i in weights:
        binary.append(tf.assign(i, tf.sign(i)))


    list_bin = []


    # initial an saver to save model
    saver = tf.train.Saver()
    config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)
    config.gpu_options.allow_growth = True
    with tf.Session(config = config) as sess:
        
        sess.run(tf.global_variables_initializer())
        summary_writer = tf.summary.FileWriter(log_save_path,sess.graph)

        # epoch = 164 
        # make sure [bath_size * iteration = data_set_number]
        k_list = []
        for i in range(num_weights):
            k_list.append(1)
        for ep in range(1,total_epoch+1):
            lr = learning_rate_schedule(ep%50)
            pre_index = 0
            train_acc = 0.0
            train_loss = 0.0
            start_time = time.time()
            print("\nepoch %d/%d:" %(ep,total_epoch))
            masks = None
            for it in range(1,iterations+1):
                batch_x = train_x[pre_index:pre_index+batch_size]
                batch_y = train_y[pre_index:pre_index+batch_size]

                batch_x = data_augmentation(batch_x)

                _, batch_loss = sess.run([train_step, cross_entropy],feed_dict={x:batch_x, y_:batch_y, keep_prob: dropout_rate, learning_rate: lr, train_flag: True, lbda:0.01, K:k_list})
                batch_acc = accuracy.eval(feed_dict={x:batch_x, y_:batch_y, keep_prob: 1.0, train_flag: True, K:k_list})

                train_loss += batch_loss
                train_acc  += batch_acc
                pre_index  += batch_size

                if it == iterations:
                    train_loss /= iterations
                    train_acc /= iterations

                    loss_, acc_  = sess.run([cross_entropy,accuracy],feed_dict={x:batch_x, y_:batch_y, keep_prob: 1.0, train_flag: True, lbda:0.01, K:k_list})
                    train_summary = tf.Summary(value=[tf.Summary.Value(tag="train_loss", simple_value=train_loss), 
                                          tf.Summary.Value(tag="train_accuracy", simple_value=train_acc)])
                    val_acc, val_loss, test_summary = run_testing(sess,ep,k_list)

                    summary_writer.add_summary(train_summary, ep)
                    summary_writer.add_summary(test_summary, ep)
                    summary_writer.flush()

                    print("iteration: %d/%d, cost_time: %ds, train_loss: %.4f, train_acc: %.4f, test_loss: %.4f, test_acc: %.4f" %(it, iterations, int(time.time()-start_time), train_loss, train_acc, val_loss, val_acc))
                    list_bin.append(sess.run(mask, feed_dict = {K:k_list}))
                    masks = sess.run(mask, feed_dict = {K:k_list})
#                     print (sess.run(weights))
                else:
                    print("iteration: %d/%d, train_loss: %.4f, train_acc: %.4f" %(it, iterations, train_loss / it, train_acc / it) , end='\r')
            if(ep%50 == 0):
                print(masks)
                k_list = update_k(masks,k_list)
            print (k_list)
        save_path = saver.save(sess, model_save_path)
        print("Model saved in file: %s" % save_path)
        print (list_bin)


======Loading data======
DataSet aready exist!
Loading ./cifar-10-batches-py/data_batch_1 : 10000.
Loading ./cifar-10-batches-py/data_batch_2 : 10000.
Loading ./cifar-10-batches-py/data_batch_3 : 10000.
Loading ./cifar-10-batches-py/data_batch_4 : 10000.
Loading ./cifar-10-batches-py/data_batch_5 : 10000.
Loading ./cifar-10-batches-py/test_batch : 10000.
Train data: (50000, 32, 32, 3) (50000, 10)
Test data : (10000, 32, 32, 3) (10000, 10)
======Load finished======
======Shuffling data======
======Prepare Finished======

epoch 1/164:
iteration: 200/200, cost_time: 181s, train_loss: 2.1806, train_acc: 0.2499, test_loss: 1.9609, test_acc: 0.3076
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

epoch 2/164:
iteration: 200/200, cost_time: 137s, train_loss: 2.0171, train_acc: 0.3110, test_loss: 1.8649, test_acc: 0.3324
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

epoch 3/164:
iteration: 200/200, cost_time: 136s, train_loss: 1.9508, train_acc: 0.3317, test_loss: 1.8004

iteration: 200/200, cost_time: 145s, train_loss: 1.3494, train_acc: 0.5404, test_loss: 1.2247, test_acc: 0.5625
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

epoch 44/164:
iteration: 200/200, cost_time: 139s, train_loss: 1.3443, train_acc: 0.5413, test_loss: 1.2223, test_acc: 0.5613
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

epoch 45/164:
iteration: 200/200, cost_time: 140s, train_loss: 1.3398, train_acc: 0.5457, test_loss: 1.2120, test_acc: 0.5649
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

epoch 46/164:
iteration: 200/200, cost_time: 141s, train_loss: 1.3393, train_acc: 0.5450, test_loss: 1.2180, test_acc: 0.5630
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

epoch 47/164:
iteration: 200/200, cost_time: 139s, train_loss: 1.3361, train_acc: 0.5472, test_loss: 1.2113, test_acc: 0.5700
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

epoch 48/164:
iteration: 200/200, cost_time: 140s, train_loss: 1.3296, train_acc: 0.5463,

iteration: 200/200, cost_time: 158s, train_loss: 1.2442, train_acc: 0.5786, test_loss: 1.1347, test_acc: 0.5921
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

epoch 87/164:
iteration: 200/200, cost_time: 161s, train_loss: 1.2444, train_acc: 0.5774, test_loss: 1.1309, test_acc: 0.5966
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

epoch 88/164:
iteration: 200/200, cost_time: 161s, train_loss: 1.2438, train_acc: 0.5767, test_loss: 1.1316, test_acc: 0.5973
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

epoch 89/164:
iteration: 200/200, cost_time: 157s, train_loss: 1.2414, train_acc: 0.5780, test_loss: 1.1311, test_acc: 0.5935
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

epoch 90/164:
iteration: 200/200, cost_time: 154s, train_loss: 1.2405, train_acc: 0.5821, test_loss: 1.1297, test_acc: 0.5933
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

epoch 91/164:
iteration: 200/200, cost_time: 152s, train_loss: 1.2375, train_acc: 0.5821,

iteration: 200/200, cost_time: 151s, train_loss: 1.2034, train_acc: 0.5900, test_loss: 1.0931, test_acc: 0.6100
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

epoch 130/164:
iteration: 200/200, cost_time: 151s, train_loss: 1.2020, train_acc: 0.5946, test_loss: 1.0951, test_acc: 0.6103
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

epoch 131/164:
iteration: 200/200, cost_time: 154s, train_loss: 1.2032, train_acc: 0.5933, test_loss: 1.0952, test_acc: 0.6101
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

epoch 132/164:
iteration: 200/200, cost_time: 153s, train_loss: 1.2017, train_acc: 0.5941, test_loss: 1.0959, test_acc: 0.6108
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

epoch 133/164:
iteration: 200/200, cost_time: 158s, train_loss: 1.1991, train_acc: 0.5933, test_loss: 1.0932, test_acc: 0.6125
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

epoch 134/164:
iteration: 200/200, cost_time: 154s, train_loss: 1.2011, train_acc: 0.

In [6]:
tf.reset_default_graph()